# 2. Defining Constraints - here for Inverse Kinematics

All methods to compute paths or configurations solve constrained optimization problems. To use them, you need to learn to define constrained optimization problems.

Some definitions:

* An objective defines either a sum-of-square cost term, or an equality constraint, or an inequality constraint in the optimization problem. An objective is defined by its type and its feature. The type can be `sos`, `eq`, or `ineq`, referring to the three cases.
* A feature is a (differentiable mapping) from the decision variable (the full path, or robot configuration) to a feature space. If the feature space is, e.g., 3-dimensional, this defines 3 sum-of-squares terms, or 2 inequality, or 3 equality constraints, one for each dimension. For instance, the feature can be the 3-dim robot hand position in the 15th time slice of a path optimization problem. If you put an equality constraint on this feature, then this adds 3 equality constraints to the overall path optimization problem.
* A feature is defined by keyword for the feature map (e.g., `pos` for position), typically by a set of frame names that tell which objects we refer to (e.g., `pr2L` for the left hand of the pr2), optionally some modifiers (e.g., a scale or target, which linearly transform the feature map), and the set of configurations IDs or time slices the feature is to be computed from (e.g., `confs=[15]` if the feat is to be computed from the 15th time slice in a path optimization problem.
* In path optimization problems, we often want to add objectives for a whole time interval rather than for specific configurations. E.g., avoid collisions from start to end. When adding objectives to the optimization problem we can specify whole intervals, with `times=[1., 2.]`, so that the objective is added to each configuration in this time interval.

TODO: give demos for all this



## Graph-structured NLP

We have a set $\{x_1,..,x_n\}$ of $n$ variables, each with domain $x_i \in \mathbb{R}^{d_i}$.

We have a set $\{\phi_1,..,\phi_K\}$ of $K$ features, each of which is a differentiable mapping $\phi_k: x_{\pi_k} \mapsto \mathbb{R}^{D_k}$. Here $\pi_k \subseteq \{1,..,n\}$ indicates a subset of the variables, and $x_{\pi_k}$ is the value of this subset of variables. In addition,  $\rho_k \in\{\texttt{ineq, eq, sos}\}$ indicates for each feature whether it is an inequality, an equality, or a sum-of-square cost objective.

This defines the mathematical program
\begin{align}
  \min_{x_1,..,x_n} \sum_{k : \rho_k=\texttt{sos}} \phi_k(x_{\pi_k})^T \phi_k(x_{\pi_k})
  ~\text{s.t.}~ \mathop\forall_{k : \rho_k=\texttt{ineq}} \phi_k(x_{\pi_k}) \le 0 ~,\quad
  \mathop\forall_{k : \rho_k=\texttt{eq}} \phi_k(x_{\pi_k}) = 0 ~,\quad
\end{align}

In summary, a graph-structured NLP is specified as a tuple $(n, \{d_i\}, K, \{\phi_k\}, \{\pi_k\}, \{\rho_k\})$.

Note: I assumed here that all cose features are sum-of-squares... That's not necessary, of course. But typically the most relevant case; and we only need the Jacobian $\partial_x \phi$ to approximate the Gauss-Newton Hessian.

Note: If all features are cost objectives, then
\begin{align}
  \prod_k \exp\{ -  \phi_k(x_{\pi_k}) \}
\end{align}
defines a factor graph. If all features are actually sum-of-square
costs and we use Gaussian belief approximations, solving the problem
(inference, which now also is MAP, which is also minimization) can be
tackled with Gaussian message passing, as in GTSAM. (But in principle,
this does not exploit more structure than a sparse matrix method to
compute Newton steps...) In this sense, a graph-structured NLP is a
generalization of factor graphs.

Note: If the structure is $k$-order Markovian (meaning that the variables $x_i$ can be sorted into a chain and cliques $\pi_k$ contain only $k+1$  consecutive variables), then the Hessian of the Augmented Lagrangian becomes banded diagonal, and Newton steps can be computed in $O(n)$. This is what I call KOMO ($k$-order Markov Optimization). BUT FOR SMALL $n$ ONE CAN TACKLE THE PROBLEM ALSO WITHOUT THE $k$-ORDER ASSUMPTION.

## Generic modifiers for all features: linear rescaling, difference/velocity, acceleration

We assume each variable $x_i$ is a world configuration, that is, the vector of all DOFs of the world in a specific time slice.

%\paragraph{\texttt{order} modifier: velocities and acceleration}

Most features are defined as functions of a single configuration $x_i$. E.g. $\phi(x_i)$ could be the 3D position of endeffector in configuration $x_i$. However, for such features we automatically also define the difference and ``acceleration'' as
\begin{align}
  \phi(x_1,x_2) &\equiv \frac{1}{\tau}(\phi(x_2) - \phi(x_1)) \\
  \phi(x_1,x_2,x_3) &\equiv \frac{1}{\tau^2}(\phi(x_1) + \phi(x_3) - 2 \phi(x_2))
\end{align}
When in the specification such a $\phi$ is indicated to depend on two configurations, it automatically means that we impose an objective on the velocity/difference; if on three configurations, then we impose an objective on the acceleration.

%\paragraph{\texttt{scale, target} modifiers: linear rescaling}

The (in)equalities always refer to zero. To constrain $\phi(x) = \textit{target}$ you can modify any defined feature by specifying a \texttt{target} and/or \texttt{scale}, which implies
\begin{align}
  \tilde \phi(x) \gets \texttt{scale} \cdot (\phi(x) - \texttt{target})
\end{align}

## Concrete features

The expressions in the left of the table declare features. All of these are functions of a \emph{single} configuration---differences and accelerations can tie multiple configurations together. Arguments typically indicate objects (actually \emph{coordinate frame identifies}), which we write as o1, o2, etc. Another typical argument is a vector (e.g., offset) interpreted in the object's coordinate frame; e.g.\ v1=$(0,0,1)$ to indicate the object's z-axis.

| feature | dim | description |
|:---:|:---:|:---:|
  | `pos(o1)` | 3 | 3D position of o1 in world coordinates |
  | `posRel(o1,o2)` | 3 | 3D position of o1 in o2 coordinates |
  | `posDiff(o1,o2)` | 3 | difference of 3D positions of o1 and o2 in world coordinates |
  | `vec(o1,v1)` | 3 | 3D vector v1 transformed from o1 to world |
  | `vecRel Diff(o1,v1,o2)` | 3 | ... |
  | `prod(o1,v1,o2,v2)` | 1 | scalar product of v1 (in o1) and v2 (in o2) |
  | `quat(o1)` | 4 | 4D quaternion of o1 in world coordinates\footnote{There is ways to handle the invariance w.r.t.\ quaternion sign properly.} |
  | `quatRel Diff(o1,o2)` | 4 | ... |
  | `pose(o1)` | 7 | 7D pose of o1 in world coordinates |
  | `poseRel Diff(o1,o2)` | 7 | ...|
| `dist(o1,o2)` | 1 | NEGATIVE distance between convex meshes o1 and o2, positive for penetration|
  | `above(o1,o2)` | ? | when all negative, o1 is above (inside support of) o2 |
  | `placed(o1,o2)` | ? | o1 is above and straightly placed on o2 |
| `coll` | ? | when negative, nothing is colliding |
  | `limits` | ? | when negative, all robot joint limits are ok |

## Dynamic/Transition features

Some features inherently refer to multiple (consecutive) configurations. Here some:



## Example Specification of a robot problem

We have two configurations $x_1, x_2$; here is a specification of features that define the NLP for grasping an object in $x_1$ and placing it on a tray in $x_2$:
```
(1,2): { sos, trans, scale:0.1 }
(1,2): { eq, pose(object, hand) }
  (1):   { eq, dist(object, hand) }
  (1):   { ineq, coll }
  (2):   { ineq, coll }
  (2):   { eq, dist(object, tray) }
  (2):   { ineq, above(object, tray) }
  (2):   { eq, vec(object, (0,0,1)), target:(0,0,1) }
```
Here, every tuple $(i,j)$ indicates an objective that depends on variables $x_i,x_j$. The @{..}@ desribes the objective. The objectives read:
* We penalize configuration difference between $x_1, x_2$ with a weak sum-of-square term
* The relative pose of object in hand has zero difference between configurations $x_1, x_2$
* The distance between object and hand is zero in configuration $x_1$
* There are no collisions in configuration $x_1$
* There are no collisions in configuration $x_2$
* The object touches tray (distance=zero) in config 2
* The object is above the tray in config 2
* The object's z-axis equals the world z-axis in config 2
